In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[4]:


#HEre how to generate sources in 2D --> apply & add modification of source position in Hook 

import inspect
from pathlib import Path
import pde 
from pde import PDE, ScalarField,FileStorage, MovieStorage, CartesianGrid,UnitGrid,PlotTracker,FieldCollection
import numpy as np
from numba import njit
import matplotlib.pyplot as plt
import pandas as pd
import ffmpeg


# In[5]:

@njit
def Brownian_dt(pos,D,dt,grid_size):
    return (pos+np.sqrt(2*D)*np.random.normal(loc=0,scale=np.sqrt(dt),size=len(pos)))%grid_size
    
def MapPos_togrid(pos,grid_spacing,grid_size):
    return np.uint(np.floor(pos/grid_spacing)%grid_size)


# In[1]:


#Define variables
#Dimer simulations
grid_size = 200 #1 mum3
grid_spacing = 0.005 # each voxel is 0.005 mum3
timestep = 1e-3
D_prot = 9.837E-2 #in mum2/ms | conversion factor is 1E14
D_subst = 4.902E-1/grid_spacing 
D_prod = 4.949E-1/grid_spacing
length = 50
rct_rate = 1 #dimer
subst_c0 = 1 # in 1 mum3: 1.1e6 particles,stress conditions: 3 mM --> 1.1E-3 particles /nm3 --> divide by 2
sysname = 'Monomer'
repno = 1 



In [3]:
#####Bug in brown: adjust scale, is currently too high #fixed
#####Bug in initial set of background 
#####bug in materiostat: **2 needs 2 be adjusted
file_write = FileStorage("2DDemo.hdf")
movie_write = MovieStorage("2DDemo.avi", vmin=-1, vmax=1)

In [4]:
grid = UnitGrid( [grid_size,grid_size], periodic=True)
background = ScalarField.from_expression(grid,'+0') 
pos1 = np.array((1,1))
pos2 = np.array((int(grid_size/2),int(grid_size/2)))
state = ScalarField(background.grid)
background.data[MapPos_togrid(pos1,grid_spacing,grid_size)[0],MapPos_togrid(pos1,grid_spacing,grid_size)[1]] = 1
background.data[MapPos_togrid(pos2,grid_spacing,grid_size)[0],MapPos_togrid(pos2,grid_spacing,grid_size)[1]] = 1

state = ScalarField(background.grid)  

def post_step_hook(state_data, t):
    global pos1
    global pos2
    #simple solution: set negative mass to 0
    #s.data = np.maximum(s.data,0)
    #'Massostat'
    s.data,p.data = s.data-(((s.data<0)*s.data).sum(0)/grid_size**2),p.data+(((s.data<0)*s.data).sum(0)/grid_size**2)
    background.data[MapPos_togrid(pos1,grid_spacing,grid_size)[0],MapPos_togrid(pos1,grid_spacing,grid_size)[1]] = 0
    background.data[MapPos_togrid(pos2,grid_spacing,grid_size)[0],MapPos_togrid(pos2,grid_spacing,grid_size)[1]] = 0
    pos1 = Brownian_dt(pos1,D_prot,timestep,grid_size)
    pos2 = Brownian_dt(pos2,D_prot,timestep,grid_size)
    background.data[MapPos_togrid(pos1,grid_spacing,grid_size)[0],MapPos_togrid(pos1,grid_spacing,grid_size)[1]]= 1
    background.data[MapPos_togrid(pos2,grid_spacing,grid_size)[0],MapPos_togrid(pos2,grid_spacing,grid_size)[1]]= 1


storage = pde.MemoryStorage()

trackers = [
    file_write.tracker(2),movie_write.tracker(1),
    "progress",  # show progress bar during simulation
    #"steady_state",  # abort when steady state is reached
    storage.tracker(interrupts=timestep),  # store data every simulation step
    pde.PlotTracker(show=False),  # show images during simulation
    # print some output every 1 real seconds:
    pde.PrintTracker(interrupts=pde.RealtimeInterrupts(duration=0.1)),
]

# define the pde
eq = PDE({"s": "D1*laplace(s) - k*source*s",
         "p": "D2*laplace(p) + k*source*s"}, 
         consts={"source": background,"D1":D_subst,"D2":D_prod,"k":rct_rate},
         post_step_hook=post_step_hook)
s,p = ScalarField(grid,label="d(Substrate)")+subst_c0,ScalarField(grid,label="d(Product)")
state = FieldCollection([s,p])
result = eq.solve(state, t_range=length, dt=timestep, scheme="runge-kutta" , tracker=trackers)
#result.plot
# In[ ]:

  0%|          | 0/50.0 [00:00<?, ?it/s]

t=0, c=0.5±0.5
t=0.004, c=0.5±0.5
t=0.005, c=0.5±0.5
t=0.018, c=0.5±0.5
t=0.053, c=0.5±0.5
t=0.111, c=0.5±0.5
t=0.185, c=0.5±0.5
t=0.276, c=0.5±0.5
t=0.376, c=0.5±0.5
t=0.481, c=0.5±0.5
t=0.59, c=0.5±0.5
t=0.703, c=0.5±0.5
t=0.814, c=0.5±0.5
t=0.92, c=0.5±0.5
t=1.03, c=0.5±0.5
t=1.135, c=0.5±0.5
t=1.244, c=0.5±0.5
t=1.358, c=0.5±0.5
t=1.472, c=0.5±0.5
t=1.585, c=0.5±0.5
t=1.7, c=0.5±0.5
t=1.815, c=0.5±0.5
t=1.93, c=0.5±0.5
t=2.045, c=0.5±0.5
t=2.152, c=0.5±0.5
t=2.263, c=0.5±0.5
t=2.375, c=0.5±0.5
t=2.489, c=0.5±0.5
t=2.604, c=0.5±0.5
t=2.718, c=0.5±0.5
t=2.832, c=0.5±0.5
t=2.948, c=0.5±0.5
t=3.063, c=0.5±0.5
t=3.17, c=0.5±0.5
t=3.281, c=0.5±0.5
t=3.394, c=0.5±0.5
t=3.506, c=0.5±0.5
t=3.62, c=0.5±0.5
t=3.733, c=0.5±0.5
t=3.848, c=0.5±0.5
t=3.962, c=0.5±0.5
t=4.077, c=0.5±0.5
t=4.157, c=0.5±0.5
t=4.253, c=0.5±0.5
t=4.358, c=0.5±0.5
t=4.468, c=0.5±0.5
t=4.58, c=0.5±0.5
t=4.695, c=0.5±0.5
t=4.809, c=0.5±0.5
t=4.924, c=0.5±0.5
t=5.038, c=0.5±0.5
t=5.145, c=0.5±0.5
t=5.25, c=0.5±0.5
t=5.357

Time mismatch: 12.001 != 12


t=12.061, c=0.5±0.499
t=12.159, c=0.5±0.499
t=12.266, c=0.5±0.499
t=12.376, c=0.5±0.499
t=12.488, c=0.5±0.499
t=12.6, c=0.5±0.499
t=12.714, c=0.5±0.499
t=12.828, c=0.5±0.499
t=12.941, c=0.5±0.499
t=13.055, c=0.5±0.499
t=13.16, c=0.5±0.499
t=13.27, c=0.5±0.499
t=13.381, c=0.5±0.499
t=13.494, c=0.5±0.499
t=13.606, c=0.5±0.499
t=13.718, c=0.5±0.499
t=13.83, c=0.5±0.499
t=13.942, c=0.5±0.499
t=14.056, c=0.5±0.499
t=14.154, c=0.5±0.499
t=14.258, c=0.5±0.499
t=14.368, c=0.5±0.499
t=14.48, c=0.5±0.499
t=14.589, c=0.5±0.499
t=14.701, c=0.5±0.499
t=14.814, c=0.5±0.499
t=14.925, c=0.5±0.499
t=15.039, c=0.5±0.499
t=15.143, c=0.5±0.499
t=15.251, c=0.5±0.499
t=15.361, c=0.5±0.499
t=15.473, c=0.5±0.499
t=15.586, c=0.5±0.499
t=15.695, c=0.5±0.499
t=15.805, c=0.5±0.499
t=15.917, c=0.5±0.499
t=16.03, c=0.5±0.499
t=16.13, c=0.5±0.499
t=16.235, c=0.5±0.499
t=16.345, c=0.5±0.499
t=16.456, c=0.5±0.499
t=16.569, c=0.5±0.499
t=16.684, c=0.5±0.499
t=16.799, c=0.5±0.499
t=16.914, c=0.5±0.499
t=17.029, c=0.5±0.

In [ ]:
thresholds = 10**np.float32(np.arange(-10,1,0.1))
tmp = np.empty((len(thresholds),(len(storage))))
for i,field in enumerate(storage):
    for j,k in enumerate(thresholds):
        tmp[j][i] = (field[1].data >= k).sum()/grid_size**3


# In[ ]:


result=np.empty((len(thresholds)))
for i,j in enumerate(tmp):
    result[i] = (np.argmax(j >= 0.5))



In [ ]:
result
pd.DataFrame({'Concentration': thresholds[:],
              '1/2 saturation reached at timestep': result[:]*timestep,
             'System': np.full((len(result)), sysname),
             'Replica': np.full((len(result)),repno)
                    }).to_csv(str(sysname+'.csv'))

In [ ]:
result

In [ ]:
storage.write_mode